In [1]:
import pandas as pd
import numpy as np
import ast
import datetime
import time

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 

### for now fold

In [15]:
# dataframe = pd.read_csv("/Users/ernestvmo/OneDrive - Universiteit Leiden/Q1/Urban Computing/Project/data/grouped_csv/data.csv", index_col=0)
dataframe = pd.read_csv("/Users/ernestvmo/OneDrive - Universiteit Leiden/Q1/Urban Computing/Project/data/FINAL DATA/dataset.csv", index_col=0)
dataframe.coordinates = dataframe.coordinates.apply(ast.literal_eval)

SAF_col = 'SAF_COUNT'
_311_col = '311_complaints_COUNT'
NYPD_col = 'nypd_complaints_COUNT'
taxi_pickup_passenger = 'passenger_count_pickup'
taxi_dropoff_passenger = 'passenger_count_dropoff'
taxi_pickup_fare_amount = 'total_amount_pickup'
taxi_dropoff_fare_amount = 'total_amount_dropoff'

In [16]:
dataframe

,datestop,coordinates,SAF_COUNT,311_complaints_COUNT,passenger_count_pickup,total_amount_pickup,passenger_count_dropoff,total_amount_dropoff,nypd_complaints_COUNT
0,2012-07-01,"(0, 20)",0.0,0.0,0.0,0.00,0.0,0.00,1
1,2012-07-01,"(0, 21)",0.0,1.0,0.0,0.00,0.0,0.00,1
2,2012-07-01,"(0, 22)",0.0,4.0,0.0,0.00,4.0,165.40,3
3,2012-07-01,"(1, 20)",0.0,3.0,0.0,0.00,0.0,0.00,1
4,2012-07-01,"(2, 20)",0.0,3.0,0.0,0.00,1.0,76.60,2
...,...,...,...,...,...,...,...,...,...
67406,2013-06-30,"(21, 17)",0.0,1.0,2.0,74.50,7.0,169.50,5
67407,2013-06-30,"(22, 8)",0.0,0.0,1.0,72.33,33.0,848.16,1
67408,2013-06-30,"(22, 10)",0.0,0.0,7.0,60.00,32.0,890.58,1
67409,2013-06-30,"(22, 12)",2.0,0.0,0.0,0.00,17.0,462.56,1


#### old

In [44]:
def find_neighbours(df):
    indices = list(df['coordinates'].values)
    neighbours_all = []

    for row in df['coordinates'].values:
        x, y = row
        neighbours = []

        if x > 0:
            NW_x = x-1
            NW_y = y
            NW = (NW_x, NW_y)
            neighbours.append(('NW', NW)) if NW in indices else ...
        if x < 23:
            NE_x = x+1
            NE_y = y
            NE = (NE_x, NE_y)
            neighbours.append(('NE', NE)) if NE in indices else ...
        if y > -1:
            NN_x = x
            NN_y = y-1
            NN = (NN_x, NN_y)
            neighbours.append(('NN', NN)) if NN in indices else ...
        if y < 22:
            NS_x = x
            NS_y = y+1
            NS = (NS_x, NS_y)
            neighbours.append(('NS', NS)) if NS in indices else ...
        neighbours_all.append(neighbours)
    return neighbours_all

In [49]:
neighbs = find_neighbours(dataframe[dataframe['datestop'] == '2012-07-01'])

In [60]:
# test = dataframe[dataframe['datestop'] == '2012-07-01']
dataframe[dataframe['datestop'] == '2012-07-01'].index[dataframe[dataframe['datestop'] == '2012-07-01']['coordinates'] == (1, 20)].to_list()[0]

3

#### current

In [17]:
new_df = dataframe.copy()
new_df['NN_SAF'] = np.nan
new_df['NN_311'] = np.nan
new_df['NN_NYPD'] = np.nan
new_df['NN_TAXI_P_PASSENGER'] = np.nan
new_df['NN_TAXI_D_PASSENGER'] = np.nan
new_df['NN_TAXI_P_FARE'] = np.nan
new_df['NN_TAXI_D_FARE'] = np.nan

new_df['NW_SAF'] = np.nan
new_df['NW_311'] = np.nan
new_df['NW_NYPD'] = np.nan
new_df['NW_TAXI_P_PASSENGER'] = np.nan
new_df['NW_TAXI_D_PASSENGER'] = np.nan
new_df['NW_TAXI_P_FARE'] = np.nan
new_df['NW_TAXI_D_FARE'] = np.nan

new_df['NE_SAF'] = np.nan
new_df['NE_311'] = np.nan
new_df['NE_NYPD'] = np.nan
new_df['NE_TAXI_P_PASSENGER'] = np.nan
new_df['NE_TAXI_D_PASSENGER'] = np.nan
new_df['NE_TAXI_P_FARE'] = np.nan
new_df['NE_TAXI_D_FARE'] = np.nan

new_df['NS_SAF'] = np.nan
new_df['NS_311'] = np.nan
new_df['NS_NYPD'] = np.nan
new_df['NS_TAXI_P_PASSENGER'] = np.nan
new_df['NS_TAXI_D_PASSENGER'] = np.nan
new_df['NS_TAXI_P_FARE'] = np.nan
new_df['NS_TAXI_D_FARE'] = np.nan

new_df['SAF_lag'] = np.nan
new_df['311_lag'] = np.nan
new_df['NYPD_lag'] = np.nan
new_df['TAXI_P_PASSENGER_lag'] = np.nan
new_df['TAXI_D_PASSENGER_lag'] = np.nan
new_df['TAXI_P_FARE_lag'] = np.nan
new_df['TAXI_D_FARE_lag'] = np.nan

# for i,row in new_df[:10].iterrows():
    

In [21]:
def get_neighbours(indices, coordinates):
    x, y = coordinates
    neighbours = []

    if x > 0:
        NW_x = x-1
        NW_y = y
        NW = (NW_x, NW_y)
        neighbours.append(('NW', NW)) if NW in indices else ...
    if x < 23:
        NE_x = x+1
        NE_y = y
        NE = (NE_x, NE_y)
        neighbours.append(('NE', NE)) if NE in indices else ...
    if y > -1:
        NN_x = x
        NN_y = y-1
        NN = (NN_x, NN_y)
        neighbours.append(('NN', NN)) if NN in indices else ...
    if y < 22:
        NS_x = x
        NS_y = y+1
        NS = (NS_x, NS_y)
        neighbours.append(('NS', NS)) if NS in indices else ...
        
    return neighbours

here
<!-- def get_dataset_after_timelag(df: pd.DataFrame, lag):
    indices = df['coordinates'].values
    df = df.sort_values(by=["datestop"], ascending=False).reset_index(drop=True)
    datelimit = datetime.datetime.strptime(df.iloc[-1]['datestop'], "%Y-%m-%d").date()
    print(datelimit)

    # for i, row in df.iterrows():
    for (i, row), _ in zip(df.iterrows(), range(10)): # for now, don't wanna do the whole df
        row_neighbours = get_neighbours(list(indices), row['coordinates'])
        date_lag = datetime.datetime.strptime(row['datestop'], "%Y-%m-%d").date() - datetime.timedelta(days=lag)

        if datelimit > date_lag:
            break
        for neighbour in row_neighbours:
            lag_row = df[df['datestop'] == str(date_lag)][df[df['datestop'] == str(date_lag)]['coordinates'] == neighbour[1]]
            if not lag_row.empty:    
                df.at[i, f'{neighbour[0]}_SAF'] = lag_row[SAF_col].values[0]
                df.at[i, f'{neighbour[0]}_311'] = lag_row[_311_col].values[0]
                df.at[i, f'{neighbour[0]}_NYPD'] = lag_row[NYPD_col].values[0]
        print(f'{i}/{len(df)}') if (i % (len(df)/10)) == 0 else ...
    
    return df.copy() -->

In [22]:
def get_dataset_after_timelag(df: pd.DataFrame, lag):
    indices = df['coordinates'].values
    df.datestop = pd.to_datetime(df.datestop).dt.date
    df = df.sort_values(by=["datestop"], ascending=False).reset_index(drop=True)
    datelimit = df.iloc[-1]['datestop']
    # print(datelimit)

    for i, row in df.iterrows():
    # for (i, row), _ in zip(df.iterrows(), range(10)): # for now, don't wanna do the whole df
        row_neighbours = get_neighbours(list(indices), row['coordinates'])
        date_lag = row['datestop'] - datetime.timedelta(days=lag)
        # print(date_lag)
        # print(row_neighbours)


        if datelimit > date_lag:
            break
        neighbour_rows = df[df['datestop'] == row['datestop']]
        for neighbour in row_neighbours:
            # neighbour_rows = df[df['datestop'] == date_lag][df[df['datestop'] == date_lag]['coordinates'] == neighbour[1]]
            neighbour_rows = neighbour_rows[neighbour_rows['coordinates'] == neighbour[1]]
            # print(neighbour_rows.values)
            if not neighbour_rows.empty:
                df.at[i, f'{neighbour[0]}_SAF'] = neighbour_rows[SAF_col].values[0]
                df.at[i, f'{neighbour[0]}_311'] = neighbour_rows[_311_col].values[0]
                df.at[i, f'{neighbour[0]}_NYPD'] = neighbour_rows[NYPD_col].values[0]
                df.at[i, f'{neighbour[0]}_TAXI_P_PASSENGER'] = neighbour_rows[taxi_pickup_passenger].values[0]
                df.at[i, f'{neighbour[0]}_TAXI_D_PASSENGER'] = neighbour_rows[taxi_dropoff_passenger].values[0]
                df.at[i, f'{neighbour[0]}_TAXI_P_FARE'] = neighbour_rows[taxi_pickup_fare_amount].values[0]
                df.at[i, f'{neighbour[0]}_TAXI_D_FARE'] = neighbour_rows[taxi_dropoff_fare_amount].values[0]

        # lag_row = df[df['datestop'] == date_lag][df[df['datestop'] == date_lag]['coordinates'] == neighbour[1]]
            # if not lag_row.empty:    
            #     df.at[i, f'{neighbour[0]}_SAF'] = lag_row[SAF_col].values[0]
            #     df.at[i, f'{neighbour[0]}_311'] = lag_row[_311_col].values[0]
            #     df.at[i, f'{neighbour[0]}_NYPD'] = lag_row[NYPD_col].values[0]

        lag_row = df[df['datestop'] == date_lag][df[df['datestop'] == date_lag]['coordinates'] == row['coordinates']]
        if not lag_row.empty:
            df.at[i, 'SAF_lag'] = lag_row[SAF_col].values[0]
            df.at[i, '311_lag'] = lag_row[_311_col].values[0]
            df.at[i, 'NYPD_lag'] = lag_row[NYPD_col].values[0]
            df.at[i, 'TAXI_P_PASSENGER_lag'] = lag_row[taxi_pickup_passenger].values[0]
            df.at[i, 'TAXI_D_PASSENGER_lag'] = lag_row[taxi_dropoff_passenger].values[0]
            df.at[i, 'TAXI_P_FARE_lag'] = lag_row[taxi_pickup_fare_amount].values[0]
            df.at[i, 'TAXI_D_FARE_lag'] = lag_row[taxi_dropoff_fare_amount].values[0]
        
        print(f'{i}/{len(df)}') if (i % (len(df)/10)) == 0 else ...

    df = df[df['datestop'] >= (datelimit + datetime.timedelta(days=lag))]
    return df.copy()

In [24]:
data = get_dataset_after_timelag(new_df, lag=1)

0/67411


In [36]:
data_7 = get_dataset_after_timelag(new_df, lag=7)

0/67411


In [37]:
data = data.fillna(0)
data_7 = data_7.fillna(0)

In [92]:
data_7

,datestop,coordinates,SAF_COUNT,311_complaints_COUNT,passenger_count_pickup,total_amount_pickup,passenger_count_dropoff,total_amount_dropoff,nypd_complaints_COUNT,NN_SAF,...,NS_TAXI_D_PASSENGER,NS_TAXI_P_FARE,NS_TAXI_D_FARE,SAF_lag,311_lag,NYPD_lag,TAXI_P_PASSENGER_lag,TAXI_D_PASSENGER_lag,TAXI_P_FARE_lag,TAXI_D_FARE_lag
0,2013-06-30,"(22, 14)",0.0,0.0,16.0,312.93,26.0,579.03,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
1,2013-06-30,"(12, 7)",0.0,0.0,52105.0,374601.91,54970.0,382690.74,9,0.0,...,0.0,0.0,0.0,4.0,38.0,9.0,53490.0,60145.0,371204.39,424270.61
2,2013-06-30,"(12, 15)",2.0,12.0,66.0,809.62,461.0,9323.87,7,0.0,...,0.0,0.0,0.0,0.0,42.0,5.0,38.0,405.0,471.89,8388.73
3,2013-06-30,"(12, 14)",9.0,10.0,86.0,757.78,923.0,15469.21,15,0.0,...,0.0,0.0,0.0,19.0,47.0,14.0,158.0,939.0,1755.31,15156.58
4,2013-06-30,"(12, 13)",9.0,30.0,585.0,5544.04,2652.0,35447.31,5,0.0,...,0.0,0.0,0.0,5.0,41.0,8.0,730.0,2552.0,7227.51,36292.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66084,2012-07-08,"(16, 2)",0.0,32.0,7.0,61.10,95.0,1903.86,11,0.0,...,0.0,0.0,0.0,5.0,32.0,8.0,23.0,119.0,255.29,1970.65
66085,2012-07-08,"(16, 1)",3.0,29.0,12.0,168.58,85.0,1619.51,18,0.0,...,0.0,0.0,0.0,4.0,33.0,8.0,13.0,69.0,35.80,1265.08
66086,2012-07-08,"(16, 0)",0.0,12.0,13.0,88.90,164.0,2540.24,1,0.0,...,0.0,0.0,0.0,0.0,5.0,3.0,3.0,124.0,33.60,2537.57
66087,2012-07-08,"(15, 15)",0.0,13.0,6.0,160.90,70.0,958.96,1,0.0,...,0.0,0.0,0.0,0.0,9.0,6.0,0.0,37.0,0.00,641.37


In [42]:
data_old = data.copy()
data_old_7 = data_7.copy()

In [91]:
data = data_old.copy(deep=True)
data_7 = data_old_7.copy(deep=True)

In [93]:
data[['x', 'y']] = pd.DataFrame(data['coordinates'].tolist(), index=data.index)
data_7[['x', 'y']] = pd.DataFrame(data_7['coordinates'].tolist(), index=data_7.index)

In [94]:
data.datestop = data.datestop.apply(lambda r: int(time.mktime(r.timetuple())))
data_7.datestop = data_7.datestop.apply(lambda r: int(time.mktime(r.timetuple())))

In [95]:
del data['coordinates'], data_7['coordinates']

In [96]:
data

,datestop,SAF_COUNT,311_complaints_COUNT,passenger_count_pickup,total_amount_pickup,passenger_count_dropoff,total_amount_dropoff,nypd_complaints_COUNT,NN_SAF,NN_311,...,NS_TAXI_D_FARE,SAF_lag,311_lag,NYPD_lag,TAXI_P_PASSENGER_lag,TAXI_D_PASSENGER_lag,TAXI_P_FARE_lag,TAXI_D_FARE_lag,x,y
0,1372543200,0.0,0.0,16.0,312.93,26.0,579.03,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,22,14
1,1372543200,0.0,0.0,52105.0,374601.91,54970.0,382690.74,9,0.0,0.0,...,0.0,3.0,26.0,21.0,64307.0,61028.0,459089.86,408074.78,12,7
2,1372543200,2.0,12.0,66.0,809.62,461.0,9323.87,7,0.0,0.0,...,0.0,3.0,25.0,11.0,57.0,390.0,660.83,8008.99,12,15
3,1372543200,9.0,10.0,86.0,757.78,923.0,15469.21,15,0.0,0.0,...,0.0,11.0,56.0,23.0,116.0,896.0,1312.94,15219.87,12,14
4,1372543200,9.0,30.0,585.0,5544.04,2652.0,35447.31,5,0.0,0.0,...,0.0,10.0,52.0,18.0,659.0,2587.0,5611.30,33921.34,12,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67214,1341180000,0.0,5.0,16.0,67.31,46.0,705.62,6,0.0,0.0,...,0.0,1.0,9.0,1.0,4.0,45.0,30.25,680.58,16,5
67215,1341180000,1.0,57.0,40.0,417.85,201.0,3148.55,13,0.0,0.0,...,0.0,3.0,40.0,21.0,57.0,267.0,526.08,4109.06,16,4
67216,1341180000,0.0,43.0,20.0,158.05,123.0,2447.49,8,0.0,0.0,...,0.0,14.0,30.0,5.0,21.0,227.0,168.26,3825.06,16,3
67217,1341180000,0.0,48.0,6.0,100.40,81.0,1648.75,14,0.0,0.0,...,0.0,5.0,32.0,8.0,23.0,119.0,255.29,1970.65,16,2


In [99]:
data.columns

Index(['datestop', 'SAF_COUNT', '311_complaints_COUNT',
       'passenger_count_pickup', 'total_amount_pickup',
       'passenger_count_dropoff', 'total_amount_dropoff',
       'nypd_complaints_COUNT', 'NN_SAF', 'NN_311', 'NN_NYPD',
       'NN_TAXI_P_PASSENGER', 'NN_TAXI_D_PASSENGER', 'NN_TAXI_P_FARE',
       'NN_TAXI_D_FARE', 'NW_SAF', 'NW_311', 'NW_NYPD', 'NW_TAXI_P_PASSENGER',
       'NW_TAXI_D_PASSENGER', 'NW_TAXI_P_FARE', 'NW_TAXI_D_FARE', 'NE_SAF',
       'NE_311', 'NE_NYPD', 'NE_TAXI_P_PASSENGER', 'NE_TAXI_D_PASSENGER',
       'NE_TAXI_P_FARE', 'NE_TAXI_D_FARE', 'NS_SAF', 'NS_311', 'NS_NYPD',
       'NS_TAXI_P_PASSENGER', 'NS_TAXI_D_PASSENGER', 'NS_TAXI_P_FARE',
       'NS_TAXI_D_FARE', 'SAF_lag', '311_lag', 'NYPD_lag',
       'TAXI_P_PASSENGER_lag', 'TAXI_D_PASSENGER_lag', 'TAXI_P_FARE_lag',
       'TAXI_D_FARE_lag', 'x', 'y'],
      dtype='object')

In [100]:
data = data[['datestop', 'x', 'y', 'SAF_COUNT', '311_complaints_COUNT',
       'passenger_count_pickup', 'total_amount_pickup',
       'passenger_count_dropoff', 'total_amount_dropoff', 'NN_SAF', 'NN_311', 'NN_NYPD',
       'NN_TAXI_P_PASSENGER', 'NN_TAXI_D_PASSENGER', 'NN_TAXI_P_FARE',
       'NN_TAXI_D_FARE', 'NW_SAF', 'NW_311', 'NW_NYPD', 'NW_TAXI_P_PASSENGER',
       'NW_TAXI_D_PASSENGER', 'NW_TAXI_P_FARE', 'NW_TAXI_D_FARE', 'NE_SAF',
       'NE_311', 'NE_NYPD', 'NE_TAXI_P_PASSENGER', 'NE_TAXI_D_PASSENGER',
       'NE_TAXI_P_FARE', 'NE_TAXI_D_FARE', 'NS_SAF', 'NS_311', 'NS_NYPD',
       'NS_TAXI_P_PASSENGER', 'NS_TAXI_D_PASSENGER', 'NS_TAXI_P_FARE',
       'NS_TAXI_D_FARE', 'SAF_lag', '311_lag', 'NYPD_lag',
       'TAXI_P_PASSENGER_lag', 'TAXI_D_PASSENGER_lag', 'TAXI_P_FARE_lag',
       'TAXI_D_FARE_lag', 'nypd_complaints_COUNT']]
data_7 = data_7[['datestop', 'x', 'y', 'SAF_COUNT', '311_complaints_COUNT',
       'passenger_count_pickup', 'total_amount_pickup',
       'passenger_count_dropoff', 'total_amount_dropoff', 'NN_SAF', 'NN_311', 'NN_NYPD',
       'NN_TAXI_P_PASSENGER', 'NN_TAXI_D_PASSENGER', 'NN_TAXI_P_FARE',
       'NN_TAXI_D_FARE', 'NW_SAF', 'NW_311', 'NW_NYPD', 'NW_TAXI_P_PASSENGER',
       'NW_TAXI_D_PASSENGER', 'NW_TAXI_P_FARE', 'NW_TAXI_D_FARE', 'NE_SAF',
       'NE_311', 'NE_NYPD', 'NE_TAXI_P_PASSENGER', 'NE_TAXI_D_PASSENGER',
       'NE_TAXI_P_FARE', 'NE_TAXI_D_FARE', 'NS_SAF', 'NS_311', 'NS_NYPD',
       'NS_TAXI_P_PASSENGER', 'NS_TAXI_D_PASSENGER', 'NS_TAXI_P_FARE',
       'NS_TAXI_D_FARE', 'SAF_lag', '311_lag', 'NYPD_lag',
       'TAXI_P_PASSENGER_lag', 'TAXI_D_PASSENGER_lag', 'TAXI_P_FARE_lag',
       'TAXI_D_FARE_lag', 'nypd_complaints_COUNT']]

In [102]:
data_7

,datestop,x,y,SAF_COUNT,311_complaints_COUNT,passenger_count_pickup,total_amount_pickup,passenger_count_dropoff,total_amount_dropoff,NN_SAF,...,NS_TAXI_P_FARE,NS_TAXI_D_FARE,SAF_lag,311_lag,NYPD_lag,TAXI_P_PASSENGER_lag,TAXI_D_PASSENGER_lag,TAXI_P_FARE_lag,TAXI_D_FARE_lag,nypd_complaints_COUNT
0,1372543200,22,14,0.0,0.0,16.0,312.93,26.0,579.03,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1
1,1372543200,12,7,0.0,0.0,52105.0,374601.91,54970.0,382690.74,0.0,...,0.0,0.0,4.0,38.0,9.0,53490.0,60145.0,371204.39,424270.61,9
2,1372543200,12,15,2.0,12.0,66.0,809.62,461.0,9323.87,0.0,...,0.0,0.0,0.0,42.0,5.0,38.0,405.0,471.89,8388.73,7
3,1372543200,12,14,9.0,10.0,86.0,757.78,923.0,15469.21,0.0,...,0.0,0.0,19.0,47.0,14.0,158.0,939.0,1755.31,15156.58,15
4,1372543200,12,13,9.0,30.0,585.0,5544.04,2652.0,35447.31,0.0,...,0.0,0.0,5.0,41.0,8.0,730.0,2552.0,7227.51,36292.56,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66084,1341698400,16,2,0.0,32.0,7.0,61.10,95.0,1903.86,0.0,...,0.0,0.0,5.0,32.0,8.0,23.0,119.0,255.29,1970.65,11
66085,1341698400,16,1,3.0,29.0,12.0,168.58,85.0,1619.51,0.0,...,0.0,0.0,4.0,33.0,8.0,13.0,69.0,35.80,1265.08,18
66086,1341698400,16,0,0.0,12.0,13.0,88.90,164.0,2540.24,0.0,...,0.0,0.0,0.0,5.0,3.0,3.0,124.0,33.60,2537.57,1
66087,1341698400,15,15,0.0,13.0,6.0,160.90,70.0,958.96,0.0,...,0.0,0.0,0.0,9.0,6.0,0.0,37.0,0.00,641.37,1


In [103]:
data.to_csv('/Users/ernestvmo/Desktop/data_lag1.csv')
data_7.to_csv('/Users/ernestvmo/Desktop/data_lag7.csv')

In [67]:
test = data.copy()

In [70]:
test.datestop = pd.to_datetime(data.datestop)

### Models

#### 7-days lag

In [12]:
# data.to_csv('/Users/ernestvmo/Desktop/data_lag1.csv')
# data_7.to_csv('/Users/ernestvmo/Desktop/data_lag7.csv')

data_7 = pd.read_csv('/Users/ernestvmo/Desktop/data_lag7.csv', index_col=0)

In [13]:
X = data_7[['datestop', 'x', 'y', 'SAF_COUNT', '311_complaints_COUNT',
       'passenger_count_pickup', 'total_amount_pickup',
       'passenger_count_dropoff', 'total_amount_dropoff', 'NN_SAF', 'NN_311', 'NN_NYPD',
       'NN_TAXI_P_PASSENGER', 'NN_TAXI_D_PASSENGER', 'NN_TAXI_P_FARE',
       'NN_TAXI_D_FARE', 'NW_SAF', 'NW_311', 'NW_NYPD', 'NW_TAXI_P_PASSENGER',
       'NW_TAXI_D_PASSENGER', 'NW_TAXI_P_FARE', 'NW_TAXI_D_FARE', 'NE_SAF',
       'NE_311', 'NE_NYPD', 'NE_TAXI_P_PASSENGER', 'NE_TAXI_D_PASSENGER',
       'NE_TAXI_P_FARE', 'NE_TAXI_D_FARE', 'NS_SAF', 'NS_311', 'NS_NYPD',
       'NS_TAXI_P_PASSENGER', 'NS_TAXI_D_PASSENGER', 'NS_TAXI_P_FARE',
       'NS_TAXI_D_FARE', 'SAF_lag', '311_lag', 'NYPD_lag',
       'TAXI_P_PASSENGER_lag', 'TAXI_D_PASSENGER_lag', 'TAXI_P_FARE_lag',
       'TAXI_D_FARE_lag', 'nypd_complaints_COUNT']].to_numpy()
y = data_7[['nypd_complaints_COUNT']].to_numpy()

In [14]:
y = y.ravel()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [16]:
X_train.shape

(52871, 45)

In [17]:
y_train.shape

(52871,)

In [18]:
from sklearn.svm import SVR
model = SVR()

In [19]:
model.fit(X_train, y_train)

In [20]:
aRMSE = mean_squared_error(y_test, model.predict(X_test), multioutput='uniform_average')

In [21]:
aRMSE

72.3758216008237

In [ ]:
score = model.score(X_test, y_test)

In [ ]:
score

0.794457812841483

##### validate erika res

In [30]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

# knn = KNeighborsRegressor()
# knn.fit(X_train, y_train)
# aRMSE = mean_squared_error(y_test, knn.predict(X_test), multioutput='uniform_average')
# aRMSE
# 39.58308972613103

# rf = RandomForestRegressor()
# rf.fit(X_train, y_train)
# aRMSE = mean_squared_error(y_test, rf.predict(X_test), multioutput='uniform_average')
# aRMSE
# 0.007417945226206687

0.007417945226206687

In [29]:
aRMSE = mean_squared_error(y_test, knn.predict(X_test), multioutput='uniform_average')
aRMSE

39.58308972613103

#### 1-day lag

In [2]:
# data.to_csv('/Users/ernestvmo/Desktop/data_lag1.csv')
# data_7.to_csv('/Users/ernestvmo/Desktop/data_lag7.csv')

data = pd.read_csv('/Users/ernestvmo/Desktop/data_lag1.csv', index_col=0)

In [3]:
X = data[['datestop', 'x', 'y', 'SAF_COUNT', '311_complaints_COUNT',
       'passenger_count_pickup', 'total_amount_pickup',
       'passenger_count_dropoff', 'total_amount_dropoff', 'NN_SAF', 'NN_311', 'NN_NYPD',
       'NN_TAXI_P_PASSENGER', 'NN_TAXI_D_PASSENGER', 'NN_TAXI_P_FARE',
       'NN_TAXI_D_FARE', 'NW_SAF', 'NW_311', 'NW_NYPD', 'NW_TAXI_P_PASSENGER',
       'NW_TAXI_D_PASSENGER', 'NW_TAXI_P_FARE', 'NW_TAXI_D_FARE', 'NE_SAF',
       'NE_311', 'NE_NYPD', 'NE_TAXI_P_PASSENGER', 'NE_TAXI_D_PASSENGER',
       'NE_TAXI_P_FARE', 'NE_TAXI_D_FARE', 'NS_SAF', 'NS_311', 'NS_NYPD',
       'NS_TAXI_P_PASSENGER', 'NS_TAXI_D_PASSENGER', 'NS_TAXI_P_FARE',
       'NS_TAXI_D_FARE', 'SAF_lag', '311_lag', 'NYPD_lag',
       'TAXI_P_PASSENGER_lag', 'TAXI_D_PASSENGER_lag', 'TAXI_P_FARE_lag',
       'TAXI_D_FARE_lag', 'nypd_complaints_COUNT']].to_numpy()
y = data[['nypd_complaints_COUNT']].to_numpy()

In [4]:
y = y.ravel()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [6]:
X_train.shape

(53775, 45)

In [7]:
y_train.shape

(53775,)

In [8]:
from sklearn.svm import SVR
model = SVR()

In [9]:
model = model.fit(X_train, y_train)

In [10]:
aRMSE = mean_squared_error(y_test, model.predict(X_test), multioutput='uniform_average')

In [11]:
aRMSE

76.53168938263086

In [76]:
score = model.score(X_test, y_test)

In [77]:
score

0.794457812841483